In [1]:
from astroquery.ipac.ned import Ned
from astropy.table import Table, Column, table, vstack
from pathlib import Path

from matplotlib import pyplot as plt

In [2]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'
path_CIGALE_phot = '../Datos/Work/CIGALE_InputPhot/'
path_CIGALE_out = '../Datos/Work/CIGALE_Output/'

groups = ['Group_01/','Group_02/','Group_03/','Group_04/','Group_05/',\
          'Group_06/','Group_07/','Group_08/','Group_09/','Group_10/']
out = ['out_1/','out_2/']

In [3]:
to_rem_o1 = [] #Rows to be deleted or removed in out_1
to_rem_o2 = [] #Rows to be deleted or removed in out_2

for i in range(1):
    CIGPhotIn = Table.read(path_CIGALE_phot+'CIGPhot_EnergyBal_'+str(i+1)+'.tbl', format='ascii')
    out_1 = Table.read(path_CIGALE_out+groups[i]+out[0]+'results.fits',format='fits')
    out_2 = Table.read(path_CIGALE_out+groups[i]+out[1]+'results.fits',format='fits')
    for j in range(len(CIGPhotIn)):
        if out_1['best.reduced_chi_square'][j] < out_2['best.reduced_chi_square'][j]:
            to_rem_o2.append(j)
        elif out_2['best.reduced_chi_square'][j] < out_1['best.reduced_chi_square'][j]:
            to_rem_o1.append(j)

print(to_rem_o1)
print(to_rem_o2)

[1, 2, 3, 4, 6, 8, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 25, 26, 27, 30, 32, 33, 34, 35, 36, 37, 38, 39, 42, 46, 47, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 71, 72, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 96, 97, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 126, 127, 129, 131, 132, 133, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 146, 147, 148, 149, 150, 152, 154, 155, 156, 159, 161, 162, 163, 167, 168, 170, 171, 172, 174, 175, 177, 178, 179, 180, 182, 183, 184, 185, 187, 188, 189, 190, 191]
[0, 5, 7, 9, 10, 11, 12, 20, 23, 28, 29, 31, 40, 41, 43, 44, 45, 48, 53, 60, 70, 73, 74, 80, 89, 95, 98, 99, 110, 111, 116, 125, 128, 130, 134, 142, 151, 153, 157, 158, 160, 164, 165, 166, 169, 173, 176, 181, 186]


In [4]:
out = vstack([out_1, out_2])
out

id,bayes.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.i,bayes.agn.i_err,bayes.agn.accretion_power,bayes.agn.accretion_power_err,bayes.agn.disk_luminosity,bayes.agn.disk_luminosity_err,bayes.agn.luminosity,bayes.agn.luminosity_err,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.FUV,bayes.FUV_err,bayes.H_2mass,bayes.H_2mass_err,bayes.IRAC1,bayes.IRAC1_err,bayes.IRAC2,bayes.IRAC2_err,bayes.IRAC3,bayes.IRAC3_err,bayes.IRAC4,bayes.IRAC4_err,bayes.IRAS1,bayes.IRAS1_err,bayes.IRAS2,bayes.IRAS2_err,bayes.IRAS3,bayes.IRAS3_err,bayes.IRAS4,bayes.IRAS4_err,bayes.J_2mass,bayes.J_2mass_err,bayes.Ks_2mass,bayes.Ks_2mass_err,bayes.MIPS1,bayes.MIPS1_err,bayes.MIPS2,bayes.MIPS2_err,bayes.MIPS3,bayes.MIPS3_err,bayes.NUV,bayes.NUV_err,bayes.PACS_blue,bayes.PACS_blue_err,bayes.PACS_green,bayes.PACS_green_err,bayes.PACS_red,bayes.PACS_red_err,bayes.PLW_ext,bayes.PLW_ext_err,bayes.PMW_ext,bayes.PMW_ext_err,bayes.PSW_ext,bayes.PSW_ext_err,bayes.WISE1,bayes.WISE1_err,bayes.WISE2,bayes.WISE2_err,bayes.WISE3,bayes.WISE3_err,bayes.WISE4,bayes.WISE4_err,bayes.g_prime,bayes.g_prime_err,bayes.i_prime,bayes.i_prime_err,bayes.r_prime,bayes.r_prime_err,bayes.u_prime,bayes.u_prime_err,bayes.z_prime,bayes.z_prime_err,best.chi_square,best.reduced_chi_square,best.agn.EBV,best.agn.Mcl,best.agn.R,best.agn.delta,best.agn.disk_type,best.agn.emissivity,best.agn.fracAGN,best.agn.fracAGN_dale2014,best.agn.i,best.agn.law,best.agn.oa,best.agn.pl,best.agn.q,best.agn.t,best.agn.temperature,best.attenuation.B_B90,best.attenuation.E_BV_factor,best.attenuation.E_BV_lines,best.attenuation.E_BVs,best.attenuation.FUV,best.attenuation.V_B90,best.attenuation.powerlaw_slope,best.attenuation.uv_bump_amplitude,best.attenuation.uv_bump_wavelength,best.attenuation.uv_bump_width,best.dust.alpha,best.nebular.f_dust,best.nebular.f_esc,best.nebular.lines_width,best.nebular.logU,best.nebular.ne,best.nebular.zgas,best.sfh.age,best.sfh.age_burst,best.sfh.age_main,best.sfh.f_burst,best.sfh.tau_burst,best.sfh.tau_main,best.stellar.age_m_star,best.stellar.imf,best.stellar.metallicity,best.stellar.old_young_separation_age,best.universe.age,best.universe.luminosity_distance,best.universe.redshift,best.agn.L_6um,best.agn.accretion_power,best.agn.disk_luminosity,best.agn.intrin_Lnu_2500A_30deg,best.agn.luminosity,best.agn.polar_dust_luminosity,best.agn.torus_dust_luminosity,best.agn.total_dust_luminosity,best.dust.luminosity,best.sfh.integrated,best.sfh.sfr,best.sfh.sfr100Myrs,best.sfh.sfr10Myrs,best.stellar.lum,best.stellar.lum_ly,best.stellar.lum_ly_old,best.stellar.lum_ly_young,best.stellar.lum_old,best.stellar.lum_young,best.stellar.m_gas,best.stellar.m_gas_old,best.stellar.m_gas_young,best.stellar.m_star,best.stellar.m_star_old,best.stellar.m_star_young,best.stellar.n_ly,best.stellar.n_ly_old,best.stellar.n_ly_young,best.FUV,best.NUV,best.u_prime,best.g_prime,best.r_prime,best.i_prime,best.z_prime,best.J_2mass,best.H_2mass,best.Ks_2mass,best.WISE1,best.IRAC1,best.IRAC2,best.WISE2,best.IRAC3,best.IRAC4,best.IRAS1,best.WISE3,best.WISE4,best.MIPS1,best.IRAS2,best.IRAS3,best.PACS_blue,best.MIPS2,best.IRAS4,best.PACS_green,best.MIPS3,best.PACS_red,best.PSW_ext,best.PMW_ext,best.PLW_ext
,,,deg,deg,W,W,W,W,W,W,W,W,W,W,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,,,mag,,,,,,,,deg,,deg,,,,K,mag,,mag,mag,mag,mag,,,nm,nm,,,,km / s,,1 / cm3,,Myr,Myr,Myr,,Myr,Myr,Myr,,,Myr,Myr,m,,W,W,W,W / Hz,W,W,W,W,W,solMass,solMass / yr,solMass / yr,solMass / yr,W,W,W,W,W,W,solMass,solMass,solMass,solMass,solMass,solMass,ph / s,ph / s,ph / s,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy,mJy
bytes26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,f

In [13]:
unique_by_name = table.unique(out, keys='id')

AttributeError: module 'astropy.table.table' has no attribute 'unique'

In [12]:
out_1['id','bayes.agn.fracAGN','bayes.agn.i','best.reduced_chi_square']

id,bayes.agn.fracAGN,bayes.agn.i,best.reduced_chi_square
,,deg,
bytes26,float64,float64,float64
M 84,0.12211827167303992,12.704999840061038,17.717618882026272
NGC 262,0.8488428007695052,9.001778236061536,5.932422804002561
IRAS 04385-0828,0.7029758990582748,39.999974368426095,11.920180311322758
IC 4553,0.10004022782016649,27.485161075931984,28.86186943669159
NGC 5252,0.761414598694148,19.332298545906653,11.577683011425723
Mrk 279,0.35649121750526613,35.75721023212308,7.878958631029752
...,...,...,...
NGC 4253,0.45758717897279977,10.450015966060858,9.27157004961008


In [ ]:
out_2['id','bayes.agn.fracAGN','bayes.agn.i','best.reduced_chi_square']

In [ ]:
o1 = out_1.remove_rows(to_rem_o1)
o2 = out_2.remove_rows(to_rem_o2)

In [ ]:
out_1

In [ ]:
out_2

### Notebook info:

In [5]:
%reload_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.9.12
IPython version      : 8.4.0

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.15.0-43-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 5.1

matplotlib: 3.5.1
astropy   : 5.1

Watermark: 2.3.1

